In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.terminal.shortcuts.filters import pass_through
from keras.src.backend.jax.random import dropout
from keras.src.layers import Dropout
from sklearn.preprocessing import LabelEncoder

In [30]:
import io

In [31]:
df=pd.read_pickle("feature_generation-data/title_and_description_tokenized_includes_polish_letters.pickle")

In [32]:
titles=[]
descriptions=[]
titles_changed=[]
descriptions_changed=[]
labels=[]
detokenized_descriptions=[]
for index,row in df.iterrows():
    if(row['category'][0]=='inżynierska' or row['category'][0]=='magisterska'):
        titles.append(row['title'])
        descriptions.append(row['description'])
        titles_changed.append(row['title_tokens'])
        descriptions_changed.append(row['description_tokens'])
        labels.append(row['category'][0])
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

for i in descriptions:
    if "inżynierskiy" in i:
        print(i)
for i in descriptions_changed:
    detokenized_descriptions.append(' '.join(str(e) for e in i))

descriptionsdetokenized=detokenized_descriptions
maxlen=1
for i in descriptionsdetokenized:
    if len(i)>maxlen:
        maxlen=len(i)


print(maxlen)
print(len(labels))

3343
4915


In [33]:
TRAINING_SIZE = 4000
VOCAB_SIZE = 10000
MAX_LENGTH = 3999
EMBEDDING_DIM = 128

In [34]:
train_titles = descriptionsdetokenized[0:TRAINING_SIZE]
test_titles = descriptionsdetokenized[TRAINING_SIZE:]

# Split the labels
train_labels = labels[0:TRAINING_SIZE]
test_labels = labels[TRAINING_SIZE:]

In [11]:
# Instantiate the vectorization layer
vectorize_layer = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE, output_sequence_length=MAX_LENGTH)

# Generate the vocabulary based on the training inputs
vectorize_layer.adapt(train_titles)

2024-12-04 10:47:20.898432: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7474 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:2b:00.0, compute capability: 8.6


In [12]:
train_sequences = vectorize_layer(train_titles)
test_sequences = vectorize_layer(test_titles)

In [13]:
train_dataset_vectorized = tf.data.Dataset.from_tensor_slices((train_sequences,train_labels))
test_dataset_vectorized = tf.data.Dataset.from_tensor_slices((test_sequences,test_labels))

In [14]:
for example in train_dataset_vectorized.take(2):
  print(example)
  print()

(<tf.Tensor: shape=(3999,), dtype=int64, numpy=array([ 79, 266,   2, ...,   0,   0,   0])>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)

(<tf.Tensor: shape=(3999,), dtype=int64, numpy=array([266,   2, 146, ...,   0,   0,   0])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)



2024-12-04 10:47:26.693967: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [15]:
SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE
BATCH_SIZE = 32

# Optimize the datasets for training
train_dataset_final = (train_dataset_vectorized
                       .cache()
                       .shuffle(SHUFFLE_BUFFER_SIZE)
                       .prefetch(PREFETCH_BUFFER_SIZE)
                       .batch(BATCH_SIZE)
                       )

test_dataset_final = (test_dataset_vectorized
                      .cache()
                      .prefetch(PREFETCH_BUFFER_SIZE)
                      .batch(BATCH_SIZE)
                      )

In [16]:
# Build the model

model = tf.keras.Sequential([
    tf.keras.Input(shape=(MAX_LENGTH,)),
    tf.keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,dropout=0.3,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,dropout=0.3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Print the model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 3999, 128)      │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 3999, 128)      │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,438,721 (5.49 MB)

 Trainable params: 1,438,721 (5.49 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer = tf.keras.optimizers.Adam(),metrics=['accuracy'])

In [18]:
num_epochs = 10

history = model.fit(train_dataset_final, epochs=num_epochs, validation_data=test_dataset_final, verbose=1)

Epoch 1/10


2024-12-04 10:47:38.904011: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101


125/125 ━━━━━━━━━━━━━━━━━━━━ 48s 347ms/step - accuracy: 0.5157 - loss: 0.6927 - val_accuracy: 0.6383 - val_loss: 0.6499
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 45s 359ms/step - accuracy: 0.6943 - loss: 0.5897 - val_accuracy: 0.5694 - val_loss: 0.7445
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 45s 363ms/step - accuracy: 0.8540 - loss: 0.3605 - val_accuracy: 0.6153 - val_loss: 0.9787
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 46s 366ms/step - accuracy: 0.9180 - loss: 0.2047 - val_accuracy: 0.5891 - val_loss: 1.3055
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 45s 361ms/step - accuracy: 0.9658 - loss: 0.1120 - val_accuracy: 0.5880 - val_loss: 1.5123
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 45s 364ms/step - accuracy: 0.9763 - loss: 0.0730 - val_accuracy: 0.5978 - val_loss: 1.7634
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 45s 360ms/step - accuracy: 0.9798 - loss: 0.0575 - val_accuracy: 0.5825 - val_loss: 1.6441
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 47s 379ms/step - accuracy: 0.9837 - loss: 0.0468 - val

In [19]:
embedding_layer = model.layers[0]

# Get the weights of the embedding layer
embedding_weights = embedding_layer.get_weights()[0]

# Print the shape. Expected is (vocab_size, embedding_dim)
print(embedding_weights.shape)

(10000, 128)


In [22]:
# Open writeable files
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

# Get the word list
vocabulary = vectorize_layer.get_vocabulary()

# Initialize the loop. Start counting at `1` because `0` is just for the padding
for word_num in range(1, len(vocabulary)):

  # Get the word associated withAttributeError the current index
  word_name = vocabulary[word_num]

  # Get the embedding weights associated with the current index
  word_embedding = embedding_weights[word_num]

  # Write the word name
  out_m.write(word_name + "\n")

  # Write the word embedding
  out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")

# Close the files
out_v.close()
out_m.close()

In [76]:
# Create a filtered DataFrame
filtered_df = df[df['category'].apply(lambda x: 'magisterska' in x or 'inżynierska' in x)].copy()

# Map the labels to binary values using .loc
filtered_df.loc[:, 'label'] = filtered_df['category'].apply(lambda x: 1 if 'magisterska' in x else 0)


# Extract features and labels
X = filtered_df['description']
y = filtered_df['label']


In [83]:
from sklearn.model_selection import train_test_split

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenization
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to fixed length
max_sequence_length = 128
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_sequence_length, padding='post')
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=max_sequence_length, padding='post')

# Convert labels to categorical (binary classification: 0 or 1)

y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)


In [87]:
# Vocabulary size and embedding dimensions
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128

# LSTM model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(max_sequence_length,)),
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, dropout=0.3, recurrent_dropout=0.3)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')  # Output layer for binary classification
])

# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Model Summary
model.summary()

# Train the model
history = model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=10,
    batch_size=32,
    verbose=1
)


/home/foka/miniconda3/envs/Guantanamo_AGH2/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_15 (Embedding)        │ (None, 128, 128)       │     4,164,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_24                │ (None, 128, 128)       │        98,816 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_25                │ (None, 64)             │        41,216 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,309,314 (16.44 MB)

 Trainable params: 4,309,314 (16.44 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 47s 353ms/step - accuracy: 0.5365 - loss: 0.6915 - val_accuracy: 0.6028 - val_loss: 0.6801
Epoch 2/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 43s 347ms/step - accuracy: 0.6894 - loss: 0.6015 - val_accuracy: 0.6431 - val_loss: 0.6476
Epoch 3/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 43s 349ms/step - accuracy: 0.9191 - loss: 0.2585 - val_accuracy: 0.6381 - val_loss: 0.9195
Epoch 4/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 44s 352ms/step - accuracy: 0.9711 - loss: 0.0897 - val_accuracy: 0.6562 - val_loss: 1.0547
Epoch 5/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 43s 351ms/step - accuracy: 0.9872 - loss: 0.0453 - val_accuracy: 0.6633 - val_loss: 1.2664
Epoch 6/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 43s 350ms/step - accuracy: 0.9911 - loss: 0.0343 - val_accuracy: 0.6371 - val_loss: 1.4722
Epoch 7/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 42s 342ms/step - accuracy: 0.9911 - loss: 0.0273 - val_accuracy: 0.6240 - val_loss: 1.6854
Epoch 8/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 44s 352ms/step - accuracy: 0.9944 - loss: 0

In [88]:
# Vocabulary size and embedding dimensions
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128

# LSTM model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(max_sequence_length,)),
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, dropout=0.3, recurrent_dropout=0.3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')  # Output layer for binary classification
])

# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Model Summary
model.summary()

# Train the model
history = model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=10,
    batch_size=32,
    verbose=1
)


/home/foka/miniconda3/envs/Guantanamo_AGH2/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_16 (Embedding)        │ (None, 128, 128)       │     4,164,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_26                │ (None, 128, 128)       │        98,816 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_27                │ (None, 64)             │        41,216 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,323,746 (16.49 MB)

 Trainable params: 4,323,746 (16.49 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 48s 352ms/step - accuracy: 0.5063 - loss: 0.6925 - val_accuracy: 0.5423 - val_loss: 0.6900
Epoch 2/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 43s 349ms/step - accuracy: 0.5751 - loss: 0.6731 - val_accuracy: 0.6260 - val_loss: 0.6360
Epoch 3/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 43s 347ms/step - accuracy: 0.8516 - loss: 0.4090 - val_accuracy: 0.6149 - val_loss: 0.8032
Epoch 4/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 44s 355ms/step - accuracy: 0.9581 - loss: 0.1554 - val_accuracy: 0.6321 - val_loss: 1.1835
Epoch 5/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 43s 351ms/step - accuracy: 0.9822 - loss: 0.0636 - val_accuracy: 0.6431 - val_loss: 1.2097
Epoch 6/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 43s 347ms/step - accuracy: 0.9906 - loss: 0.0409 - val_accuracy: 0.6351 - val_loss: 1.2974
Epoch 7/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 43s 345ms/step - accuracy: 0.9887 - loss: 0.0528 - val_accuracy: 0.6321 - val_loss: 1.4477
Epoch 8/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 43s 351ms/step - accuracy: 0.9928 - loss: 0

In [51]:
train_descriptions = descriptions[0:TRAINING_SIZE]
test_descriptions = descriptions[TRAINING_SIZE:]

# Split the labels
train_labels = labels[0:TRAINING_SIZE]
test_labels = labels[TRAINING_SIZE:]

In [52]:
vectorize_layer = tf.keras.layers.TextVectorization(max_tokens=15000, output_sequence_length=MAX_LENGTH)

# Generate the vocabulary based on the training inputs
vectorize_layer.adapt(train_descriptions)

In [53]:
train_sequences = vectorize_layer(train_descriptions)
test_sequences = vectorize_layer(test_descriptions)

In [54]:
train_dataset_vectorized = tf.data.Dataset.from_tensor_slices((train_sequences,train_labels))
test_dataset_vectorized = tf.data.Dataset.from_tensor_slices((test_sequences,test_labels))

In [55]:
SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE
BATCH_SIZE = 64

# Optimize the datasets for training
train_dataset_final = (train_dataset_vectorized
                       .cache()
                       .shuffle(SHUFFLE_BUFFER_SIZE)
                       .prefetch(PREFETCH_BUFFER_SIZE)
                       .batch(BATCH_SIZE)
                       )

test_dataset_final = (test_dataset_vectorized
                      .cache()
                      .prefetch(PREFETCH_BUFFER_SIZE)
                      .batch(BATCH_SIZE)
                      )

In [58]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(MAX_LENGTH,)),
    tf.keras.layers.Embedding(15000, EMBEDDING_DIM),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,dropout=0.3,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,dropout=0.3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Print the model summary
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 3999, 128)      │     1,920,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ (None, 3999, 128)      │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_7 (Bidirectional) │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,078,721 (7.93 MB)

 Trainable params: 2,078,721 (7.93 MB)

 Non-trainable params: 0 (0.00 B)

In [59]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer = tf.keras.optimizers.Adam(),metrics=['accuracy'])
num_epochs = 10

history = model.fit(train_dataset_final, epochs=num_epochs, validation_data=test_dataset_final, verbose=1)

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 29s 396ms/step - accuracy: 0.5207 - loss: 0.6928 - val_accuracy: 0.3650 - val_loss: 0.6980
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 24s 389ms/step - accuracy: 0.5437 - loss: 0.6860 - val_accuracy: 0.6011 - val_loss: 0.6519
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 24s 387ms/step - accuracy: 0.7827 - loss: 0.4937 - val_accuracy: 0.5366 - val_loss: 0.8747
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 25s 394ms/step - accuracy: 0.9180 - loss: 0.2245 - val_accuracy: 0.5760 - val_loss: 1.0912
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 25s 394ms/step - accuracy: 0.9642 - loss: 0.1060 - val_accuracy: 0.5781 - val_loss: 1.5925
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 25s 392ms/step - accuracy: 0.9806 - loss: 0.0662 - val_accuracy: 0.5760 - val_loss: 1.5187
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 25s 390ms/step - accuracy: 0.9884 - loss: 0.0445 - val_accuracy: 0.6000 - val_loss: 1.9216
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 25s 392ms/step - accuracy: 0.9894 - loss: 0.0411 - val_accu

In [60]:
embedding_layer = model.layers[0]

# Get the weights of the embedding layer
embedding_weights = embedding_layer.get_weights()[0]

# Print the shape. Expected is (vocab_size, embedding_dim)
print(embedding_weights.shape)

(15000, 128)


In [61]:
# Open writeable files
out_v = io.open('vecs2.tsv', 'w', encoding='utf-8')
out_m = io.open('meta2.tsv', 'w', encoding='utf-8')

# Get the word list
vocabulary = vectorize_layer.get_vocabulary()

# Initialize the loop. Start counting at `1` because `0` is just for the padding
for word_num in range(1, len(vocabulary)):

  # Get the word associated withAttributeError the current index
  word_name = vocabulary[word_num]

  # Get the embedding weights associated with the current index
  word_embedding = embedding_weights[word_num]

  # Write the word name
  out_m.write(word_name + "\n")

  # Write the word embedding
  out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")

# Close the files
out_v.close()
out_m.close()